# Testes acerca da nossa carteira

In [35]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
from io import StringIO

In [36]:
data_mad_dy = pd.read_excel("/Users/pedroafmelo/Documents/ufpb/aulas_jorge/projeto_final/data/dy_mad.xlsx", index_col=0)
data_mad_pr = pd.read_excel("/Users/pedroafmelo/Documents/ufpb/aulas_jorge/projeto_final/data/price_mad.xlsx", index_col=0)
data_dividendos = pd.read_excel("/Users/pedroafmelo/Documents/ufpb/aulas_jorge/projeto_final/data/dy_mean.xlsx", index_col=0)

for df in [data_mad_dy, data_mad_pr, data_dividendos]:
    df.reset_index(inplace=True)
    df.rename(columns={"index": "ticker"}, inplace=True)

In [37]:
data_dividendos

,ticker,dy_medio
0,TAEE11,0.099173
1,VIVT3,0.060711
2,BBSE3,0.070420
3,SANB11,0.064221
4,EGIE3,0.053694
5,ITSA4,0.063601
6,BBAS3,0.076782
7,BRAP4,0.142050
8,CMIG4,0.099123
9,CPLE6,0.073391


## Extraindo pesos iniciais

In [38]:
url = "https://www.b3.com.br/pt_br/noticias/b3-lanca-primeiro-indice-derivado-do-ibovespa-b3-para-acompanhar-empresas-que-mais-pagam-dividendos.htm"

response = get(url)

soup = BeautifulSoup(response.content, "html.parser")

table_content = soup.find_all("table")[0]

## Lendo tabela e validando valores das instruções

In [39]:
alocacoes = pd.read_html(StringIO(str(table_content)))[0]
alocacoes.columns = ["ticker", "empresa", "posicao"]

alocacoes = alocacoes.assign(
    posicao = lambda x: (x["posicao"].str.slice(0,3).astype(float))/ 100
)

alocacoes = alocacoes.drop(19)
alocacoes.index = range(alocacoes.shape[0])
alocacoes

,ticker,empresa,posicao
0,TAEE11,Taesa,0.058
1,VIVT3,Telefônica Brasil,0.058
2,BBSE3,BB Seguridade,0.056
3,SANB11,Banco Santander (Brasil),0.055
4,EGIE3,Engie Brasil,0.054
5,ITSA4,Itausa AS,0.053
6,BBSA3,Banco do Brasil,0.053
7,BRAP4,Bradespar,0.053
8,CMIG4,Companhia Energética de Minas Gerais S.A,0.053
9,CPLE6,Copel,0.048


## Retorno de referência com DY

In [40]:
weight_vector = np.array(alocacoes["posicao"])
returns_vector = np.array(data_dividendos["dy_medio"])
dy_mad_vector = np.array(data_mad_dy["dy_mad"])
price_mad_vector = np.array(data_mad_pr["pr_mad"])

dy_indice = float(sum(weight_vector * returns_vector)) 
dy_mad_indice = float(sum(weight_vector * dy_mad_vector))
pr_mad_indice = float(sum(weight_vector * price_mad_vector))

print(f"MAD DY Indice: {dy_mad_indice}")
print(f"MAD Price Indice: {pr_mad_indice}")
print(f"Returns DY Indice: {dy_indice}")

MAD DY Indice: 0.03645803582090973
MAD Price Indice: 0.21232501573052617
Returns DY Indice: 0.07881475386047396


## Realizando a otimização

Agora que temos os valores das nossas restrições, a ideia é que consigamos otimizar o nosso retorno a partir de uma alocação ideal em cada ativo!

In [41]:
import pyomo.environ as pyo

In [42]:
finance = ["BBSE3", "SANB11", "ITSA4", "BBAS3", "BRAP4", "CIEL3"]

energy = ["TAEE11", "EGIE3", "CMIG4", "CPLE6", "CPFE3"]

others = ["VIVT3", "CSNA3", "VALE3", "GGBR4", "VBBR3", "CYRE3", "GOAU4", "PETR4", "MRFG3"]

all_assets = finance + energy + others

weight_vector = np.array(alocacoes["posicao"])
returns_dict = {ticker:dy for ticker, dy in zip(data_dividendos["ticker"], data_dividendos["dy_medio"])}
returns_vector = np.array(data_dividendos["dy_medio"])
dy_mad_dict = {ticker:dy for ticker, dy in zip(data_mad_dy["ticker"], data_mad_dy["dy_mad"])}
dy_mad_vector = np.array(data_mad_dy["dy_mad"])
pr_mad_dict = {ticker:dy for ticker, dy in zip(data_mad_pr["ticker"], data_mad_pr["pr_mad"])}
pr_mad_vector = np.array(data_mad_pr["pr_mad"])

dy_indice = float(sum(weight_vector * returns_vector)) 
dy_mad_indice = float(sum(weight_vector * dy_mad_vector))
pr_mad_indice = float(sum(weight_vector * price_mad_vector))


In [43]:
returns_dict

{'TAEE11': 0.09917346147225437,
 'VIVT3': 0.06071122771388349,
 'BBSE3': 0.07042023323461288,
 'SANB11': 0.06422064178422149,
 'EGIE3': 0.05369417832852186,
 'ITSA4': 0.06360062153628165,
 'BBAS3': 0.07678151901147534,
 'BRAP4': 0.142049627629272,
 'CMIG4': 0.0991225596634715,
 'CPLE6': 0.07339078284078271,
 'CPFE3': 0.0766069763268431,
 'CSNA3': 0.08945154159222635,
 'VALE3': 0.0855581616797152,
 'GGBR4': 0.06306261691186421,
 'VBBR3': 0.06529939650388372,
 'CYRE3': 0.05209458237746234,
 'GOAU4': 0.0975068057783175,
 'CIEL3': 0.05191659652542868,
 'PETR4': 0.1952574233553306,
 'MRFG3': 0.06030852726122493}

In [46]:
all_assets

['BBSE3',
 'SANB11',
 'ITSA4',
 'BBAS3',
 'BRAP4',
 'CIEL3',
 'TAEE11',
 'EGIE3',
 'CMIG4',
 'CPLE6',
 'CPFE3',
 'VIVT3',
 'CSNA3',
 'VALE3',
 'GGBR4',
 'VBBR3',
 'CYRE3',
 'GOAU4',
 'PETR4',
 'MRFG3']

In [45]:
model = pyo.ConcreteModel()

model.weight = pyo.Var(all_assets, domain=pyo.NonNegativeReals)

model.retorno = pyo.Objective(expr=sum([model.weight[asset] * returns_dict[asset] 
                                        for asset in all_assets]),
                              sense=pyo.maximize)

model.cons = pyo.ConstraintList()


model.cons.add(expr=sum([model.weight[weight] for weight in model.weight]) == 1)

for weight in model.weight:
    model.cons.add(expr= model.weight[weight] <= 0.20)

model.cons.add(sum([model.weight[asset] for asset in model.weight if asset in finance]) <= 0.50)
model.cons.add(sum([model.weight[asset] for asset in model.weight if asset in energy]) <= 0.50)
model.cons.add(sum([model.weight[asset] for asset in model.weight if asset in others]) <= 0.50)

model.cons.add(expr=sum([model.weight[asset] * dy_mad_dict[asset] for asset in all_assets]) <= dy_mad_indice)
model.cons.add(expr=sum([model.weight[asset] * pr_mad_dict[asset] for asset in all_assets]) <= pr_mad_indice)

solver = pyo.SolverFactory("glpk")

result = solver.solve(model)

model.display()

Model unknown

  Variables:
    weight : Size=20, Index={BBSE3, SANB11, ITSA4, BBAS3, BRAP4, CIEL3, TAEE11, EGIE3, CMIG4, CPLE6, CPFE3, VIVT3, CSNA3, VALE3, GGBR4, VBBR3, CYRE3, GOAU4, PETR4, MRFG3}
        Key    : Lower : Value              : Upper : Fixed : Stale : Domain
         BBAS3 :     0 :                0.2 :  None : False : False : NonNegativeReals
         BBSE3 :     0 : 0.0942056570156143 :  None : False : False : NonNegativeReals
         BRAP4 :     0 :                0.0 :  None : False : False : NonNegativeReals
         CIEL3 :     0 :                0.0 :  None : False : False : NonNegativeReals
         CMIG4 :     0 :                0.2 :  None : False : False : NonNegativeReals
         CPFE3 :     0 :                0.0 :  None : False : False : NonNegativeReals
         CPLE6 :     0 :                0.0 :  None : False : False : NonNegativeReals
         CSNA3 :     0 :                0.0 :  None : False : False : NonNegativeReals
         CYRE3 :     0 :    

In [48]:
model.cons[-2]()

KeyError: "Index '-2' is not valid for indexed component 'cons'"